# Part-of-Speech Tagging 

In this lesson, we're going to learn about the textual analysis methods *part-of-speech tagging* and *keyword extraction*. These methods will help us computationally parse sentences and better understand words in context.

---

```{epigraph}
[Charles] Babbage, who called [Ada Lovelace] the “enchantress of numbers,” once wrote that she “has thrown her magical spell around the most abstract of Sciences and has grasped it with a force which few masculine intellects (in our own country at least) could have exerted over it.

-- Claire Cain Miller, ["Ada Lovelace,"](https://www.nytimes.com/interactive/2018/obituaries/overlooked-ada-lovelace.html) *NYT Overlooked Obituaries*
```

In [16]:
#Set some display options for the visualizer
options = {"compact": True, "distance": 50, "color": "yellow", "bg": "black", "font": "Gill Sans"}

displacy.render(document, style="dep", options=options)

---

## Why is Part-of-Speech Tagging Useful?

I don't mean to go all [Language Nerd](https://xkcd.com/1443/) on you, but parts of speech are important. Even if they seem kind of boring. *Parts of speech* are the grammatical units of language — such as (in English) nouns, verbs, adjectives, adverbs, pronouns, and prepositions. Each of these parts of speech plays a different role in a sentence.

<img src="https://imgs.xkcd.com/comics/language_nerd.png" >


By computationally identifying parts of speech, we can start computationally exploring *syntax*, the relationship between words — rather than only focusing on words in isolation, as we did with tf-idf. Though parts of speech may seem pedantic, they help computers (and us) crack at that ever-elusive abstract noun — *meaning*. 

## spaCy and Natural Language Processing (NLP)

To computationally identify parts of speech, we're going to use the natural language processing library spaCy. For a more extensive introduction to NLP and spaCy, see the previous lesson.

To parse sentences, spaCy relies on machine learning models that were trained on large amounts of labeled text data. The English-language spaCy model that we're going to use in this lesson was trained on an annotated corpus called ["OntoNotes"](https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf): 2 million+ words drawn from "news, broadcast, talk shows, weblogs, usenet newsgroups, and conversational telephone speech," which were meticulously tagged by a group of researchers and professionals for people's names and places, for nouns and verbs, for subjects and objects, and much more.

## Install spaCy

To use spaCy, we first need to install the library.

In [ ]:
!pip install -U spacy

## Import Libraries

Then we're going to import `spacy` and `displacy`, a special spaCy module for visualization.

In [1]:
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
pd.set_option("max_rows", 400)
pd.set_option("max_colwidth", 400)

We're also going to import the `Counter` module for counting nouns, verbs, adjectives, etc., and the `pandas` library for organizing and displaying data (we're also changing the pandas default max row and column width display setting).

## Download Language Model

Next we need to download the English-language model (`en_core_web_sm`), which will be processing and making predictions about our texts. This is the model that was trained on the annotated "OntoNotes" corpus. You can download the `en_core_web_sm` model by running the cell below:

In [6]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


*Note: spaCy offers [models for other languages](https://spacy.io/usage/models#languages) including German, French, Spanish, Portuguese, Italian, Dutch, Greek, Norwegian, and Lithuanian. Languages such as Russian, Ukrainian, Thai, Chinese, Japanese, Korean and Vietnamese don't currently have their own NLP models. However, spaCy offers language and tokenization support for many of these language with external dependencies — such as [PyviKonlpy](https://github.com/konlpy/konlpy) for Korean or [Jieba](https://github.com/fxsjy/jieba) for Chinese.*

## Load Language Model

Once the model is downloaded, we need to load it with `spacy.load()` and assign it to the variable `nlp`.

In [2]:
nlp = spacy.load('en_core_web_sm')

## Create a Processed spaCy Document

Whenever we use spaCy, our first step will be to create a processed spaCy `document` with the loaded NLP model `nlp()`. Most of the heavy NLP lifting is done in this line of code. After processing, the `document` object will contain tons of juicy language data — named entities, sentence boundaries, parts of speech — and the rest of our work will be devoted to accessing this information.

To test out spaCy's part-of-speech tagging, we'll begin by processing a sample sentence from Ada Lovelace's obituary:

> "[Charles] Babbage, who called [Ada Lovelace] the “enchantress of numbers,” once wrote that
she “has thrown her magical **spell** around the most **abstract** of Sciences and has grasped
it with a **force** which few masculine intellects (in our own country at least) could have exerted over it.

This sentence makes for an interesting example because it is syntactically complex and because it includes contains difficultly ambiguous words such as "spell," "abstract," and "force."

In [6]:
sample = """She “has thrown her magical spell around the most abstract of Sciences."""

In [10]:
document = nlp(sample)

## spaCy Part-of-Speech Tagging

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| ADJ   | adjective                 | big, old, green, incomprehensible, first      |
| ADP   | adposition                | in, to, during                                |
| ADV   | adverb                    | very, tomorrow, down, where, there            |
| AUX   | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ  | conjunction               | and, or, but                                  |
| CCONJ | coordinating conjunction  | and, or, but                                  |
| DET   | determiner                | a, an, the                                    |
| INTJ  | interjection              | psst, ouch, bravo, hello                      |
| NOUN  | noun                      | girl, cat, tree, air, beauty                  |
| NUM   | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART  | particle                  | ’s, not,                                      |
| PRON  | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT | punctuation               | ., (, ), ?                                    |
| SCONJ | subordinating conjunction | if, while, that                               |
| SYM   | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :), 😝             |
| VERB  | verb                      | run, runs, running, eat, ate, eating          |
| X     | other                     | sfpksdpsxmsa                                  |
| SPACE | space                     |                                               |


Above is a POS chart taken from [spaCy's website](https://spacy.io/api/annotation#named-entities), which shows the different parts of speech that spaCy can identify as well as their corresponding labels. To quickly see spaCy's POS tagging in action, we can use the [spaCy module `displacy`](https://spacy.io/usage/visualizers#ent) on our sample `document` with the `style=` parameter set to "dep" (short for dependency parsing):

In [15]:
#Set some display options for the visualizer
options = {"compact": True, "distance": 90, "color": "yellow", "bg": "black", "font": "Gill Sans"}

displacy.render(document, style="dep", options=options)

As you can see, spaCy has correctly identified that "spell" and "force" are nouns in our sample sentence:

In [1667]:
for token in document:
    if token.pos_ == "NOUN":
        print(token, token.pos_)

spell NOUN
force NOUN
intellects NOUN
country NOUN


But if we look at the same words in a different context — in a sentence that I made up — spaCy can identify when these words have changed  grammatical roles and meanings.

> You shouldn't **force** someone to learn how to **spell** Babbage. They just need practice. You can't **abstract** it.

In [1671]:
document = nlp("You shouldn't force someone to learn how to spell Babbage. They just need practice. You can't abstract it.")

In [1672]:
for token in document:
    if token.pos_ == "VERB":
        print(token, token.pos_)

force VERB
learn VERB
spell VERB
need VERB
abstract VERB


Where previously spaCy had identified "force" and "spell" as nouns, here spaCy correctly identifies the words "force," "spell," and "abstract" as verbs.

## Get Part-Of-Speech Tags

To get part of speech tags for every word in a document, we have to iterate through all the tokens in the document and pull out the `.pos_` attribute for each token. We can get even finer-grained dependency information with the attribute `.dep_`.


In [1674]:
for token in document:
    print(token.text, token.pos_, token.dep_)

You PRON nsubj
should AUX aux
n't ADV neg
force VERB ROOT
someone NOUN dobj
to PART aux
learn VERB xcomp
how ADV advmod
to PART aux
spell VERB xcomp
Babbage PROPN dobj
. PUNCT punct
They PRON nsubj
just ADV advmod
need VERB ROOT
practice NOUN dobj
. PUNCT punct
You PRON nsubj
ca AUX aux
n't ADV neg
abstract VERB ROOT
it PRON dobj
. PUNCT punct


## Practicing with *Lost in the City*

For the rest of this lesson, we're going to explore Edward P. Jones's short story collection *Lost in the City*.

<img src="https://mybinder.org/static/images/logo_social.png" width="150" align="left" > *If you're using this Jupyter notebook in Binder (in the cloud), please uncomment the cell below and work with only the first story from _Lost in the City_. The Binder notebook is currently having issues loading the entire collection.*

In [ ]:
#file = "../texts/literature/Lost-in-the-City_Stories/01-The-Girl-Who-Raised-Pigeons.txt"
#document = nlp(open(file).read())

In [1677]:
filepath = "../texts/literature/Jones-Lost-in-The-City.txt"
document = nlp(open(filepath, encoding="utf-8").read())

## Get Adjectives

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| ADJ   | adjective                 | big, old, green, incomprehensible, first      |

To extract and count the adjectives in *Lost in the City*, we will follow the same model as above, except we'll add an `if` statement that will pull out words only if their POS label matches "ADJ."

```{admonition} Python Review!
:class: pythonreview
While we demonstrate how to extract parts of speech in the sections below, we're also going to reinforce some integral Python skills. Notice how we use `for` loops and `if` statements to `.append()` specific words to a list. Then we count the words in the list and make a pandas dataframe from the list.
```

Here we make a list of the adjectives identified in *Lost in the City*:

In [1468]:
adjs = []
for token in document:
    if token.pos_ == 'ADJ':
        adjs.append(token.text)

In [1469]:
adjs

['many',
 'vivid',
 'simple',
 'front',
 'few',
 'most',
 'least',
 'wooden',
 'few',
 'brown',
 'old',
 'few',
 'old',
 'eager',
 'same',
 'frightened',
 'furious',
 'colorful',
 'thick',
 'extended',
 'old',
 'same',
 'gray',
 'dead',
 'different',
 'radiant',
 'black',
 'impossible',
 'old',
 'most',
 'more',
 'own',
 'great',
 'such',
 'such',
 'difficult',
 'old',
 'few',
 'good',
 'obedient',
 'good',
 'most',
 'dead',
 'first',
 'long',
 'hopeful',
 'long',
 'half',
 'asleep',
 'most',
 'more',
 'welcome',
 'dead',
 'dead',
 'such',
 'little',
 'more',
 'good',
 'young',
 'blue',
 'crazy',
 'young',
 'wet',
 'own',
 'close',
 'certain',
 'clear',
 'uttered',
 'double',
 'next',
 'vacant',
 'front',
 'last',
 'rainy',
 'bigger',
 'much',
 'glad',
 'small',
 'top',
 'other',
 'most',
 'first',
 'pretty',
 'last',
 'third',
 'other',
 'other',
 'other',
 'other',
 'small',
 'old',
 'last',
 'old',
 'few',
 'right',
 'young',
 'whole',
 'lighter',
 'fluttering',
 'good',
 'good',
 '

Then we count the unique adjectives in this list with the `Counter()` module:

In [1470]:
adjs_tally = Counter(adjs)

In [1471]:
adjs_tally.most_common()

[('other', 150),
 ('old', 142),
 ('more', 131),
 ('first', 106),
 ('good', 103),
 ('few', 101),
 ('little', 94),
 ('next', 66),
 ('small', 65),
 ('last', 64),
 ('white', 57),
 ('same', 52),
 ('second', 50),
 ('long', 49),
 ('much', 49),
 ('own', 48),
 ('new', 46),
 ('right', 44),
 ('whole', 42),
 ('dead', 41),
 ('most', 36),
 ('young', 34),
 ('only', 34),
 ('many', 32),
 ('several', 31),
 ('big', 31),
 ('bad', 30),
 ('such', 28),
 ('front', 26),
 ('black', 26),
 ('retarded', 26),
 ('dark', 25),
 ('late', 22),
 ('blue', 21),
 ('nice', 21),
 ('older', 21),
 ('high', 21),
 ('alone', 21),
 ('sorry', 21),
 ('best', 20),
 ('large', 20),
 ('back', 20),
 ('real', 20),
 ('able', 20),
 ('empty', 19),
 ('enough', 18),
 ('ready', 18),
 ('less', 18),
 ('surprised', 18),
 ('asleep', 17),
 ('green', 17),
 ('better', 17),
 ('loud', 17),
 ('damn', 16),
 ('afraid', 16),
 ('full', 16),
 ('open', 15),
 ('tiny', 15),
 ('happy', 15),
 ('5th', 15),
 ('least', 14),
 ('close', 14),
 ('easy', 14),
 ('quiet', 14

Then we make a dataframe from this list:

In [1476]:
df = pd.DataFrame(adjs_tally.most_common(), columns=['character', 'count'])
df[:100]

,character,count
0,other,150
1,old,142
2,more,131
3,first,106
4,good,103
5,few,101
6,little,94
7,next,66
8,small,65
9,last,64


## Get Nouns

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| NOUN  | noun                      | girl, cat, tree, air, beauty                  |

To extract and count nouns, we can follow the same model as above, except we will change our `if` statement to check for POS labels that match "NOUN".

In [1475]:
nouns = []
for token in document:
    if token.pos_ == 'NOUN':
        nouns.append(token.text)

nouns_tally = Counter(nouns)

df = pd.DataFrame(nouns_tally.most_common(), columns=['noun', 'count'])
df[:100]

,noun,count
0,man,297
1,mother,258
2,woman,251
3,time,246
4,father,182
5,day,182
6,way,163
7,people,156
8,door,146
9,“,145


## Get Verbs

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| VERB  | verb                      | run, runs, running, eat, ate, eating          |

To extract and count works of art, we can follow a similar-ish model to the examples above. This time, however, we're going to make our code even more economical and efficient (while still changing our `if` statement to match the POS label "VERB").

```{admonition} Python Review!
:class: pythonreview
We can use a [*list comprehension*](https://melaniewalsh.github.io/Intro-Cultural-Analytics/Python/More-Lists-Loops.html#List-Comprehensions) to get our list of verbs in a single line of code! Closely examine the first line of code below:
```

In [1474]:
verbs = [token.text for token in document if token.pos_ == 'VERB']

verbs_tally = Counter(verbs)

df = pd.DataFrame(verbs_tally.most_common(), columns=['verb', 'count'])
df[:100]

,verb,count
0,was,1377
1,had,1119
2,said,822
3,would,511
4,did,306
5,be,288
6,were,281
7,do,273
8,could,255
9,been,252


# Keyword Extraction

## Get Sentences with Keyword

spaCy can also identify sentences in a document. To access sentences, we can iterate through `document.sents` and pull out the `.text` of each sentence.

We can use spaCy's sentence-parsing capabilities to extract sentences that contain particular keywords, such as in the function below.

With the function `find_sentences_with_keyword()`, we will iterate through `document.sents` and pull out any sentence that contains a particular "keyword." Then we will display these sentence with the keywords bolded.

In [1089]:
import regex
from IPython.display import Markdown, display

In [1678]:
def find_sentences_with_keyword(keyword, document):
    
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        sentence = sentence.text
        
        #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
        if keyword.lower() in sentence.lower():
            
            #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
            sentence = re.sub('\n', ' ', sentence)
            sentence = re.sub(f"{keyword}", f"**{keyword}**", sentence, flags=re.IGNORECASE)
            
            display(Markdown(sentence))

In [1679]:
find_sentences_with_keyword(keyword="mother", document=document)

Her father had known Miles long before the girl was born, before the thought to marry her **mother** had even crossed his mind.

The men found her in the kitchen, her head buried in the lap of Miles’s **mother**, her arms tight around the waist of the old woman, who had been sitting at the table having Sunday lunch with her husband.  

Buster,” Miles’s **mother** said to him,

One morning, toward four o’clock, they cut open her stomach and pulled out the child only moments after Clara died, **mother** and daughter passing each other as if along a corridor, one into death, the other into life.  

There was a song about trains that kept rumbling in his head as the night wore on, a song his **mother** would sing when he was a boy.  

...

When I went back home to visit that first time and told my **mother** all about my livin in Washington, she fell into such a cry, like maybe I had managed to make it to heaven without dyin.

She had not played another tape since the one about her **mother**

This white family my **mother** worked for, they had a son married and gone to live in Baltimore.

So he wrote to his **mother** and she asked my **mother** and my **mother** asked me about goin to live in Baltimore.

Hadn’t ever been kissed, hadn’t ever been anything, but here I was goin farther from home than my **mother** and father put together….

## Get Keyword in Context

We can also find out about a keyword's more immediate context — its neighboring words to the left and right — and we can fine-tune our search with POS tagging.

To do so, we will first create a list of what's called *ngrams*. "Ngrams" are any sequence of *n* tokens in a text. They're an important concept in computational linguistics and NLP. (Have you ever played with [Google's *Ngram* Viewer](https://books.google.com/ngrams)?)

Below we're going to make a list of *bigrams*, that is, all the two-word combinations from *Lost in the City*. We're going to use these bigrams to find the neighboring words that appear alongside particular keywords.

In [1680]:
#Make a list of tokens and POS labels from document if the token is a word 
tokens_and_labels = [(token.text, token.pos_) for token in document if token.is_alpha]

In [1687]:
#Make a function to get all two-word combinations
def get_bigrams(word_list, number_consecutive_words=2):
    
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)
    
    #Loop through numbers from 0 to the (slightly adjusted) length of your word list
    for word_index in range(adj_length_of_word_list):
        
        #Index the list at each number, grabbing the word at that number index as well as N number of words after it
        ngram = word_list[word_index : word_index + number_consecutive_words]
        
        #Append this word combo to the master list "ngrams"
        ngrams.append(ngram)
        
    return ngrams

In [1682]:
bigrams = get_bigrams(tokens_and_labels)

Let's take a peek at the bigrams:

In [1686]:
bigrams[5:20]

[[('Her', 'DET'), ('father', 'NOUN')],
 [('father', 'NOUN'), ('would', 'VERB')],
 [('would', 'VERB'), ('say', 'VERB')],
 [('say', 'VERB'), ('years', 'NOUN')],
 [('years', 'NOUN'), ('later', 'ADV')],
 [('later', 'ADV'), ('that', 'ADP')],
 [('that', 'ADP'), ('she', 'PRON')],
 [('she', 'PRON'), ('had', 'VERB')],
 [('had', 'VERB'), ('dreamed', 'VERB')],
 [('dreamed', 'VERB'), ('that', 'DET')],
 [('that', 'DET'), ('part', 'NOUN')],
 [('part', 'NOUN'), ('of', 'ADP')],
 [('of', 'ADP'), ('it', 'PRON')],
 [('it', 'PRON'), ('that', 'ADP')],
 [('that', 'ADP'), ('she', 'PRON')]]

Now that we have our list of bigrams, we're going to make a function `get_neighbor_words()`. This function will return the most frequent words that appear next to a particular keyword. The function can also be fine-tuned to return neighbor words that match a certain part of speech by changing the `pos_label` parameter.

In [1688]:
def get_neighbor_words(keyword, bigrams, pos_label = None):
    
    neighbor_words = []
    keyword = keyword.lower()
    
    for bigram in bigrams:
        
        #Extract just the lowercased words (not the labels) for each bigram
        words = [word.lower() for word, label in bigram]        
        
        #Check to see if keyword is in the bigram
        if keyword in words:
            
            for word, label in bigram:
                
                #Now focus on the neighbor word, not the keyword
                if word.lower() != keyword:
                    #If the neighbor word matches the right pos_label, append it to the master list
                    if label == pos_label or pos_label == None:
                        neighbor_words.append(word.lower())
    
    return Counter(neighbor_words).most_common()

In [1689]:
get_neighbor_words("mother", bigrams)

[('my', 86),
 ('her', 75),
 ('his', 39),
 ('and', 27),
 ('had', 24),
 ('said', 18),
 ('the', 18),
 ('would', 12),
 ('was', 9),
 ('day', 8),
 ('your', 7),
 ('she', 6),
 ('their', 6),
 ('has', 5),
 ('anita', 4),
 ('told', 4),
 ('in', 4),
 ('died', 3),
 ('church', 3),
 ('says', 3),
 ('i', 3),
 ('as', 3),
 ('who', 3),
 ('only', 3),
 ('hand', 3),
 ('thought', 3),
 ('lived', 3),
 ('he', 3),
 ('a', 3),
 ('lies', 3),
 ('lydia', 3),
 ('ebbie', 3),
 ('miles', 2),
 ('but', 2),
 ('to', 2),
 ('does', 2),
 ('when', 2),
 ('is', 2),
 ('asks', 2),
 ('before', 2),
 ('wanted', 2),
 ('put', 2),
 ('of', 2),
 ('with', 2),
 ('old', 2),
 ('once', 2),
 ('at', 2),
 ('marvella', 2),
 ('lap', 2),
 ('following', 2),
 ('asked', 2),
 ('own', 2),
 ('happy', 2),
 ('it', 2),
 ('woodrow', 2),
 ('did', 1),
 ('things', 1),
 ('returns', 1),
 ('slaps', 1),
 ('tells', 1),
 ('shakes', 1),
 ('that', 1),
 ('what', 1),
 ('looks', 1),
 ('takes', 1),
 ('answers', 1),
 ('slowly', 1),
 ('continues', 1),
 ('offers', 1),
 ('presents',

In [1691]:
get_neighbor_words("mother", bigrams, pos_label='ADJ')

[('own', 2),
 ('closed', 1),
 ('proud', 1),
 ('other', 1),
 ('asleep', 1),
 ('happy', 1),
 ('such', 1),
 ('best', 1),
 ('old', 1),
 ('good', 1)]

## Your Turn!

Try out `find_sentences_with_keyword()` and `get_neighbor_words` with your own keywords of interest.

In [ ]:
find_sentences_with_keyword(keyword="YOUR KEY WORD", document=document)

In [ ]:
get_neighbor_words(keyword="YOUR KEY WORD", bigrams, pos_label=None)